**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Generación de texto usando caracteres y redes recurrentes  

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Referencias

1. Tensorflow, [Text generation with a RNN](https://www.tensorflow.org/tutorials/text/text_generation)
2. Ralf C. Staudemeyer and Eric Rothstein Morris,[Understanding LSTM a tutorial into Long Short-Term Memory Recurrent Neural Networks*, arxiv, September 2019](https://arxiv.org/pdf/1909.09586.pdf)
3. karpathy, [The Unreasonable Effectiveness of Recurrent Neural Networks](  http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

## Contenido

* [Introducción](#Introducción)
* [Importa módulos requeridos](#Importa-módulos-requeridos)
* [Descarga los datos](#Descarga-los-datos)
* [Crea  el alfabeto](#Crea-el-alfabeto)
* [Crea los diccionarios ](#Crea-los-diccionarios )
* [Transforma el texto en un arreglo de caracteres](#Transforma-el-texto-en-un-arreglo-de-caracteres)
* [Creación de datos de entrenamiento y etiquetas](#Creación-de-datos-de-entrenamiento-y-etiquetas)
* [Construcción del Modelo](#Construcción-del-Modelo)
* [Prueba del modelo ](#Prueba-del-modelo)
* [Entrenamiento](#Entrenamiento)
* [Generación de texto](#Generación-de-texto)
* [Entrenamiento personalizado](#Entrenamiento-personalizado)

# Introducción

Este cuaderno es una adaptación del tutorial de tensorflow [Text generation with a RNN](https://www.tensorflow.org/tutorials/text/text_generation).

Se muestra cómo generar texto usando un RNR basado en caracteres. Trabajaremos con un conjunto de datos de los escritos de Shakespeare preparados por Andrej Karpathy en su blog sobre [La irrazonable efectividad de las redes neuronales recurrente]( http://karpathy.github.io/2015/05/21/rnn-effectiveness/). 


Dada una secuencia de caracteres a partir de estos datos ("Shakespear"), se entrena un modelo para predecir el siguiente carácter en la secuencia ("e"). 


Se pueden generar secuencias de texto más largas llamando al modelo repetidamente.


## Importa módulos requeridos

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

print("Versión de Tensorflow: ", tf.__version__)

Versión de Tensorflow:  2.1.0


## Descarga los datos

Obtiene el path en donde están los datos y los descarga

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Lee los datos

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


### Una mirada a los primeros 250  caracters

In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
text[:250]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n'

## Crea  el alfabeto

Este alfabeto es una lista que contiene las letras mayúscula y munúsculas, y algunos caracteres especiales, incluyendo el salto de línea '\n'.

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [8]:
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

## Crea los diccionarios 

- caracter a índice
- índice a caracter

In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [10]:
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [17]:
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

## Transforma el texto en un arreglo de caracteres

In [18]:
text_as_int = np.array([char2idx[c] for c in text])

In [19]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [20]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


### Demostración del uso de los diccionarios

La función *repr* convierte el argumento en una expresión imprimible(cuando es posible).

In [21]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### La tarea de predicción


Dado un caracter, o una secuencia de caracteres, ¿cuál es el próximo caracter más probable? Esta es la tarea en la que vamos a entrenar al modelo. 


La entrada al modelo será una secuencia de caracteres, y entrenamos al modelo para predecir la salida, el siguiente carácter en cada paso de tiempo.

Dado que los RNR mantienen un estado interno que depende de los elementos vistos anteriormente, dados todos los caracteres calculados hasta este momento, ¿cuál es el siguiente carácter?


## Creación de datos de entrenamiento y etiquetas 

El texto se divide  en secuencias de entrenamiento. Cada secuencia de entrada contendrá  una longitud *seq_length* de caracteres del texto.

Para cada secuencia de entrada, las etiquetas correspondientes contienen la misma longitud de texto, excepto que se desplaza un carácter a la derecha.

Así que primero se divide el texto en trozos de *seq_length + 1*. Por ejemplo, digamos que *seq_length* es 3 y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia de destino "ola".


In [25]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

Luego se usa la función *tf.data.Dataset.from_tensor_slices* para convertir el vector de texto en una secuencia de índices de caracteres. Un tensor de enteros.

In [26]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

F
i
r
s
t
 
C
i
t
i


In [27]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

El método por lotes nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.


In [28]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [29]:
sequences

<BatchDataset shapes: (101,), types: tf.int64>

In [30]:
for item in sequences.take(5):
  print(repr(idx2char[item.numpy()]))

array(['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n',
       ':', '\n', 'B', 'e', 'f', 'o', 'r', 'e', ' ', 'w', 'e', ' ', 'p',
       'r', 'o', 'c', 'e', 'e', 'd', ' ', 'a', 'n', 'y', ' ', 'f', 'u',
       'r', 't', 'h', 'e', 'r', ',', ' ', 'h', 'e', 'a', 'r', ' ', 'm',
       'e', ' ', 's', 'p', 'e', 'a', 'k', '.', '\n', '\n', 'A', 'l', 'l',
       ':', '\n', 'S', 'p', 'e', 'a', 'k', ',', ' ', 's', 'p', 'e', 'a',
       'k', '.', '\n', '\n', 'F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't',
       'i', 'z', 'e', 'n', ':', '\n', 'Y', 'o', 'u', ' '], dtype='<U1')
array(['a', 'r', 'e', ' ', 'a', 'l', 'l', ' ', 'r', 'e', 's', 'o', 'l',
       'v', 'e', 'd', ' ', 'r', 'a', 't', 'h', 'e', 'r', ' ', 't', 'o',
       ' ', 'd', 'i', 'e', ' ', 't', 'h', 'a', 'n', ' ', 't', 'o', ' ',
       'f', 'a', 'm', 'i', 's', 'h', '?', '\n', '\n', 'A', 'l', 'l', ':',
       '\n', 'R', 'e', 's', 'o', 'l', 'v', 'e', 'd', '.', ' ', 'r', 'e',
       's', 'o', 'l', 'v', 'e', 'd', '.', '\n', '\n', '


Para cada secuencia, duplíquela y cámbiela para formar el texto de entrada y de destino utilizando el método  map para aplicar una función simple a cada lote. Es  similar a la función apply de R.

In [31]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [32]:
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [33]:
dataset.take(2)

<TakeDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [34]:
for item in dataset.take(2):
  print(item)

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59])>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
array([47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
       53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52,
       63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51,
       43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54,
       43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57,
       58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1])>)
(<tf.Tensor: shap

In [35]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [36]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### Creación de lotes de entrenamiento

Usamos `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, necesitamos mezclar los datos y empaquetarlos en lotes.

In [37]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Construcción del Modelo


Use `tf.keras.Sequential` para definir el modelo. Para este sencillo ejemplo, se utilizan tres capas para definir nuestro modelo:

In [38]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [27]:
vocab_size

65

In [41]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [42]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


<figure>
<center>
<img src="../Imagenes/text_generation_training.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de generación de texto</p>
</figcaption>
</figure>

Fuente: [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation)

## Prueba del modelo 


Ahora ejecute el modelo para ver que se comporta como se esperaba.

Primero verifique la forma de la salida:

In [43]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


En el ejemplo anterior, la longitud de secuencia de la entrada es `100` pero el modelo puede ejecutarse en entradas de cualquier longitud.

El dataset de entrenamiento que tiene lotes de tamaño 64 * 100, los cuales tiene orden aleatorio (ver celda 249 se extrae El primer lote actual y se pasa por el modelo.

El modelo tiene de momento los pesos inciales. El lote de datos pasa por el modelo y devuelve un tensor de tamaño (64, 100, 65).

Como el vocabulario tien 65 caracteres y cada sequencia tiene tamaño 100, el modelo predice el siguiente caractrer para cada caracter en la entrada.

La predicción funciona así:

Por cada caracter de entrada, se predice el siguiente caractaer así: el modelo asigna un valor numérico a cada elemento en el vocabulario. Si se eligiera el máximo el caracter seleccionado sería simplemente el carater con el valor mas grande.

Ya por fuera del modelo.

El siguiente es el vector de predicciones del siguiente caracter para todos los caracteres de la primera secuencia en el primer bloque.

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


Para obtener predicciones reales del modelo, necesitamos tomar muestras de la distribución de salida, para obtener índices de caracteres reales. Esta distribución está definida por los logits sobre el vocabulario de los caracteres.

Nota: Es mejor muestrear  esta distribución,  que tomar el *argmax* para evitar  que el modelo quede atascado en un bucle.

Probemos para el primer ejemplo en el lote:

In [45]:
  example_batch_predictions[0]

<tf.Tensor: shape=(100, 65), dtype=float32, numpy=
array([[ 0.0098305 ,  0.00348802, -0.00275722, ...,  0.00632911,
        -0.00936237, -0.00554749],
       [ 0.01394573,  0.00106732, -0.01497596, ...,  0.00198129,
        -0.0045165 ,  0.00952342],
       [ 0.01688722,  0.00932702, -0.00521472, ...,  0.00950518,
         0.00519089, -0.00446027],
       ...,
       [ 0.00468424, -0.0112329 , -0.01078923, ..., -0.00687163,
        -0.00174603, -0.00093643],
       [ 0.01254865, -0.00709624, -0.00906933, ...,  0.00107603,
        -0.00896124, -0.00605359],
       [ 0.01664659, -0.00633432,  0.0001511 , ...,  0.01343223,
        -0.01652833, -0.00117041]], dtype=float32)>

Esto valores se transforman con logits para obtener probabilidades. Entonces se normaliza a la una distribución categórica, es decir se usa softmax. Con esto, se genera una muestra de la variable categórica para seleccionar el caracter.

El siguiente es el vector de predicciones del sigiente caracter para todos los caracteres de  la primera secuencia en el primer bloque.

Vamos a predecir los caracteres como indicamos.

In [46]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[59],
       [32],
       [20],
       [ 7],
       [40],
       [17],
       [23],
       [30],
       [ 6],
       [22],
       [53],
       [ 2],
       [14],
       [33],
       [10],
       [54],
       [59],
       [20],
       [63],
       [40],
       [46],
       [ 5],
       [28],
       [60],
       [14],
       [24],
       [57],
       [ 4],
       [49],
       [17],
       [15],
       [55],
       [ 3],
       [20],
       [13],
       [48],
       [50],
       [35],
       [ 1],
       [50],
       [ 2],
       [ 4],
       [64],
       [61],
       [49],
       [18],
       [ 1],
       [23],
       [12],
       [20],
       [32],
       [16],
       [33],
       [51],
       [25],
       [44],
       [52],
       [ 3],
       [47],
       [32],
       [13],
       [44],
       [47],
       [39],
       [47],
       [49],
       [42],
       [33],
       [42],
       [ 9],
       [64],
       [35],
       [10],
   

In [34]:
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([58, 59, 49, 20, 30, 28, 44, 57, 56,  8, 23, 40, 11, 59, 16, 43,  0,
       18,  8, 48, 25, 27, 48, 14,  7,  3, 49, 23, 43, 44, 42,  6, 47, 53,
        2, 21,  3, 56,  5, 63,  2, 22, 35, 17, 11, 28,  1, 48, 25, 36,  0,
       35, 42, 28, 38, 18, 11, 63, 32, 63, 57,  1, 43,  7, 49, 20, 45,  1,
       29,  0, 34, 18, 54,  0, 54, 44, 38,  6,  8, 24, 11, 24, 58, 14, 62,
       44, 16, 25, 36, 20, 34, 37,  6,  0, 42, 23,  9, 48, 43, 39])

Esto nos da, en cada paso de tiempo, una predicción del siguiente índice de caracteres:

In [35]:
sampled_indices

array([58, 59, 49, 20, 30, 28, 44, 57, 56,  8, 23, 40, 11, 59, 16, 43,  0,
       18,  8, 48, 25, 27, 48, 14,  7,  3, 49, 23, 43, 44, 42,  6, 47, 53,
        2, 21,  3, 56,  5, 63,  2, 22, 35, 17, 11, 28,  1, 48, 25, 36,  0,
       35, 42, 28, 38, 18, 11, 63, 32, 63, 57,  1, 43,  7, 49, 20, 45,  1,
       29,  0, 34, 18, 54,  0, 54, 44, 38,  6,  8, 24, 11, 24, 58, 14, 62,
       44, 16, 25, 36, 20, 34, 37,  6,  0, 42, 23,  9, 48, 43, 39])

Esta salida significa que el caracter predicho como el siguiente para el primer caracter es el caracter 58 y asi...

Los textos lucen así actualmente

Decodifíquelos para ver el texto predicho por este modelo no entrenado:

In [36]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' so my care\nTo have you royally appointed as if\nThe scene you play were mine. For instance, sir,\nTha'

Next Char Predictions: 
 "tukHRPfsr.Kb;uDe\nF.jMOjB-$kKefd,io!I$r'y!JWE;P jMX\nWdPZF;yTys e-kHg Q\nVFp\npfZ,.L;LtBxfDMXHVY,\ndK3jea"


## Entrenamiento

En este punto, el problema puede tratarse como un problema de clasificación estándar. Dado el estado RNN anterior, y la entrada en este paso de tiempo, predice la clase del siguiente carácter.

Se define una función de pérdida adecuada para este caso. Esta función recibe los targets (labels) y las predicciones La función calcula la distribución a partir de las predicciones y genera para cada caracter un valor predicho. Esto es lo que se necesita para calcular la entropía cruzada en este caso (promedio). Este es el valor retornado. En el ejemplo, como entra una secuencia y se predicen 100 caracteres hay 100 distribuciones para el siguiente caracter. Una por cada caracter en la entrada. En total hay un lote de 64 secuencias. Luego la función de pérdida se basa en el promedio de  64∗100=6400  entropías cruzadas dispersas.

### Adjuntar un optimizador y una función de pérdida

La función de pérdida estándar `tf.keras.losses.sparse_categorical_crossentropy` funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que nuestro modelo devuelve logits, debemos establecer el indicador `from_logits`.

In [47]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1738386



Configure el procedimiento de entrenamiento utilizando el método `tf.keras.Model.compile`. Usaremos `tf.keras.optimizers.Adam` con argumentos predeterminados y la función de pérdida.

In [48]:
model.compile(optimizer='adam', loss=loss)

### Configuración de checkpoints

Utilice un `tf.keras.callbacks.ModelCheckpoint` para asegurarse de que los puntos de control se guarden durante el entrenamiento. Se crea un directorio en el cual se guardará checkpoints con el estado del modelos. Los pesos son guardados allí para uso posterior

In [49]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Ejecuta el entrenamiento

In [50]:
EPOCHS=10

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

En history queda al información de loss, y acurracy para revisión.

## Generación de texto

### Restaura el último  checkpoint

En esta sección vamos a tomar un pre-entranamiento. 

Usaremos los pesos almacenados, pero ahora vamos a cambiar el modelo para recibir lotes de tamaño 1. Es decir vamos a recibir una secuencia para hacer la predicción a partir de esa secuencia.


Debido a la forma en que se pasa el estado RNN de un paso a otro, el modelo solo acepta un tamaño de lote fijo una vez construido.

Para ejecutar el modelo con un tamaño de lote diferente, necesitamos reconstruir el modelo y restaurar los pesos desde el punto de control.

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


### El bucle de predicción

El siguiente bloque de código genera el texto:

* Comienza eligiendo una cadena de inicio, inicializando el estado de la  RNR y configurando el número de caracteres a generar.

* Obtenga la distribución de predicción del siguiente carácter utilizando la cadena de inicio y el estado de la  RNR.

* Luego, use una distribución categórica para calcular el índice del carácter predicho. Use este caracter predicho como nuestra próxima entrada al modelo.

* El estado RNN devuelto por el modelo se retroalimenta al modelo para que ahora tenga más contexto, en lugar de una sola palabra. Después de predecir la siguiente palabra, los estados RNN modificados se retroalimentan nuevamente en el modelo, que es cómo aprende a medida que obtiene más contexto de las palabras predichas previamente.

La siguiente es la linea que define la capa GRU arriba en la definición del modelo.

In [ ]:
#tf.keras.layers.GRU(rnn_units,  return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')

El parámetro *stateful* determina si los valores de estado recurrente deben matenerse (True) o no, cuando se pasa al siguiente caracter. En este caso, al mantenerse esos valores, se mantiene la memoria de la secuencia inicial y de los caracteres que van siendo generados. 

Por eso, solamente se pasa en el primer paso la secuencia de entrada completa. Desde el segundo paso solamente se pasa el nuevo caracter (que es el predicho). Con esta predicción y la memoria del resto de la secuencia anterior mantenida en estado recurrente se hace la siguiente predicción y así sucesivamente tomado la última secuencia pasada por la capa.

Para pasar la última secuencia tratada en la capa GRU, se usa el parámetro *return_sequences*. Por defecto es falso, es decir se omite en la salida la última secuencia pasada por la capa GRU. Aquí hemos colocado *return_sequences=True*, para pasar únicamente el último carater predicho.




Al observar el texto generado, verá que el modelo sabe cuándo colocar mayúsculas, hacer párrafos e imita un vocabulario de escritura similar a Shakespeare. Con el pequeño número de épocas de entrenamiento, aún no ha aprendido a formar oraciones coherentes.

<figure>
<center>
<img src="../Imagenes/text_generation_sampling.png" width="800" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Muestreo  de palabras</p>
</figcaption>
</figure>

Fuente: [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation)

La siguiente función implementa el ciclo de predicción

In [46]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states() # reset the recurrent units 
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

### Notas.
1. *reset_states* borra solo los estados recurrentes de la red. Vale la pena mencionar que dependiendo de si la opción *stateful = True* se configuró en la red, el comportamiento de esta función podría ser diferente. Si no está configurado, todos los estados se restablecen automáticamente después de cada cálculo por lotes en su red (por ejemplo, después de llamar a *fit, predict y evaluate* también). Si no es así, debe llamar a reset_states cada vez que desee que las llamadas de modelo consecutivas sean independientes.
2. *tf.expand_dims* devuelve un tensor con una dimensión adicional insertada en el eje de índice.

In [47]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: they are toward up comfort,
As when thou comest: you show'd mementing,
She did unpeening
too:
Their lastless gay of my wife, as he be togs and pide
And not become our son.

Provost:
I brought thee. How our dath have to
Forguit as pinisly wanting fair Nadua,
Or honourable garther of my request of your affection
Thy general and strange of a goadensfleshy enough their instrument:
Consent, and nothing craves them; for my consumate me
To the day a shrewful boy:
Eacle! pardonly; for, upure rume that cannot be

NORTHUMBERLAND:
This one that should confluct thee here the time
From what my treeching frames in my
soltue garlands. But what make hassing
That rather content a wife, queen. War it, make toon
To nou quaken.

KANGARIT:
Ay, so Gentle queen,
Or any other dismay ent-hatred be the first blood icud tale!

CLEON MARAARANA:
Good night! phe undonefully: Ill periel.

LEONTES:
No!
There was thy daughter?

First Senator:
What trefts it were, for honour, thou wilt
Flatter Kate, and hither s

Lo más fácil que puedes hacer para mejorar los resultados es entrenarlo por más tiempo (prueba `EPOCHS = 30`).

También puede experimentar con una cadena de inicio diferente, o intentar agregar otra capa RNN para mejorar la precisión del modelo, o ajustar el parámetro de temperatura para generar predicciones más o menos aleatorias.

## Entrenamiento personalizado



El procedimiento de entrenamiento anterior es simple, pero no le da mucho control.

Entonces, ahora que ha visto cómo ejecutar el modelo manualmente, descomprimimos el ciclo de entrenamiento e implementémoslo nosotros mismos. Esto proporciona un punto de partida si, por ejemplo, se implementa _ aprendizaje curricular_ para ayudar a estabilizar la salida de bucle abierto del modelo.

Usaremos `tf.GradientTape` para rastrear los gradientes. Puede obtener más información sobre este enfoque leyendo el [eager execution guide](https://www.tensorflow.org/guide/eager).

El procedimiento funciona de la siguiente manera:

* Primero, inicialice el estado RNN. Hacemos esto llamando al método `tf.keras.Model.reset_states`.

* Luego, repita el conjunto de datos (lote por lote) y calcule las * predicciones * asociadas con cada una.

* Abra un `tf.GradientTape`, y calcule las predicciones y pérdidas en ese contexto.

* Calcular los gradientes de la pérdida con respecto a las variables del modelo utilizando el método `tf.GradientTape.grads`.

* Finalmente, dé un paso hacia abajo utilizando el método `tf.train.Optimizer.apply_gradients` del optimizador.

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

- [Regresar al inicio](Contenido)